<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/349_WDO_RoleEvolution_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a **very strong role evolution layer** — and importantly, it completes the narrative arc of the agent:
from *risk*, to *gaps*, to *learning*, to **structural change**.

You’ve avoided the most common trap here: treating role evolution as a vague “future state.” Instead, you’ve made it **measurable, prioritized, and operational**.

---

# Workforce Development Orchestrator — Role Evolution Analysis Utilities

This module is responsible for analyzing **how roles should change as automation reshapes work**. It turns abstract ideas about “future roles” into **concrete, prioritized transformation plans** that leadership can evaluate and act on.

Rather than assuming change is always urgent or always beneficial, the utilities apply **explicit criteria** to determine *when* and *how* a role should evolve.

---

## 1. Readiness Is Quantified, Not Assumed

The `calculate_role_readiness_score` function determines **how prepared a role is for evolution** based on real signals, not intuition.

The score combines:

* Overall automation risk for the role
* Severity of that risk (low / medium / high)
* The nature of the proposed evolution (augmented vs transformed)
* The presence of affected employees

This ensures that role evolution is:

* Triggered by evidence
* Scaled by urgency
* Grounded in workforce reality

A role with no employees or no risk signal does not get artificially prioritized.

---

## 2. Automation Risk Drives Timing, Not Panic

Automation risk is used as a **timing signal**, not a fear signal.

* High-risk roles increase readiness
* Medium-risk roles progress steadily
* Low-risk roles are deliberately deprioritized

This avoids the common executive mistake of trying to “transform everything at once” and instead supports **sequenced, manageable change**.

---

## 3. Evolution Type Matters

The utilities explicitly distinguish between:

* **Augmented roles** (AI supports human work)
* **Transformed roles** (work structure fundamentally changes)

This distinction affects readiness scoring and downstream recommendations, ensuring that:

* Incremental change is not over-managed
* Deep transformation receives appropriate attention

Not all change requires the same level of intervention.

---

## 4. Analysis Produces Implementation-Ready Outputs

The `analyze_role_evolution` function converts abstract evolution concepts into **execution-ready insights**, including:

* Which employees are affected
* Which tasks will be automated or augmented
* Which new skills are required
* A readiness score
* A clear implementation priority (low / medium / high)

This allows leaders to ask:

> “Which role transformations should we actually fund this quarter?”

And get a defensible answer.

---

## 5. Recommendations Are Practical, Not Aspirational

Recommendations are generated conditionally based on what is changing:

* Task automation leads to workload reallocation guidance
* New skill requirements trigger upskilling recommendations
* Full transformations prompt change management warnings

These are **operational signals**, not vision statements.

---

## 6. Organization-Wide Analysis Is Consistent and Scalable

The `analyze_all_role_evolutions` function applies the same logic uniformly across all roles.

This ensures:

* Fair prioritization
* Comparable readiness scores
* Portfolio-level workforce planning

No role is treated as “special” unless the data supports it.

---

## Why This Design Works for Leaders

From a leadership perspective, this module ensures:

* Role change is justified by risk and readiness
* Transformation effort is sized appropriately
* Workforce impact is visible before decisions are made
* Change can be phased, not rushed

From a system design perspective, it ensures:

* Deterministic behavior
* Clear extension points
* Easy testing
* Policy-aligned decision-making

---

## Architectural Takeaway

This module demonstrates a critical principle of responsible AI-driven transformation:

> **Role evolution should be a managed transition, not a reactive response.**

By quantifying readiness and separating augmentation from transformation, the orchestrator helps leaders modernize work **without destabilizing the organization**.




In [ ]:
"""Role evolution analysis utilities for Workforce Development Orchestrator

Following the pattern: Utilities implement, nodes orchestrate.
These utilities analyze role evolution opportunities.
"""

from typing import Dict, List, Any, Optional
from config import WorkforceDevelopmentOrchestratorConfig


def calculate_role_readiness_score(
    role_evolution: Dict[str, Any],
    employees_by_role: Dict[str, List[Dict[str, Any]]],
    automation_risk_analysis: List[Dict[str, Any]]
) -> float:
    """Calculate readiness score for role evolution"""
    role_id = role_evolution.get("role_id")
    employees = employees_by_role.get(role_id, [])

    if not employees:
        return 0.0

    # Find automation risk for this role
    role_risk = next(
        (r for r in automation_risk_analysis if r.get("role_id") == role_id),
        None
    )

    if not role_risk:
        return 0.5  # Default if no risk data

    # Base score from automation risk (higher risk = higher readiness)
    risk_score = role_risk.get("overall_risk_score", 0.0)

    # Boost if role has high automation risk
    if role_risk.get("risk_level") == "high":
        readiness = min(1.0, risk_score * 1.2)
    elif role_risk.get("risk_level") == "medium":
        readiness = risk_score
    else:
        readiness = risk_score * 0.8

    # Boost if evolution type is "transformed" (more urgent)
    if role_evolution.get("evolution_type") == "transformed":
        readiness = min(1.0, readiness * 1.1)
    elif role_evolution.get("evolution_type") == "augmented":
        readiness = min(1.0, readiness * 1.05)

    return round(readiness, 2)


def analyze_role_evolution(
    role_evolution: Dict[str, Any],
    employees_by_role: Dict[str, List[Dict[str, Any]]],
    roles_lookup: Dict[str, Dict[str, Any]],
    automation_risk_analysis: List[Dict[str, Any]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """Analyze a single role evolution opportunity"""
    role_id = role_evolution.get("role_id")
    role = roles_lookup.get(role_id)
    employees = employees_by_role.get(role_id, [])

    # Calculate readiness score
    readiness_score = calculate_role_readiness_score(
        role_evolution,
        employees_by_role,
        automation_risk_analysis
    )

    # Determine implementation priority
    if readiness_score >= 0.7:
        implementation_priority = "high"
    elif readiness_score >= 0.4:
        implementation_priority = "medium"
    else:
        implementation_priority = "low"

    # Get affected employees
    affected_employees = [emp["employee_id"] for emp in employees]

    # Generate recommendations
    recommendations = []
    if role_evolution.get("automated_tasks"):
        recommendations.append(
            f"Automate {len(role_evolution['automated_tasks'])} tasks to free up time for higher-value work"
        )
    if role_evolution.get("new_skills_required"):
        recommendations.append(
            f"Upskill {len(affected_employees)} employees in {len(role_evolution['new_skills_required'])} new skills"
        )
    if role_evolution.get("evolution_type") == "transformed":
        recommendations.append(
            "Role transformation requires comprehensive change management and training program"
        )

    analysis = {
        "role_id": role_id,
        "role_name": role.get("role_name", "") if role else "",
        "evolution_id": role_evolution.get("evolution_id", ""),
        "evolution_type": role_evolution.get("evolution_type", ""),
        "description": role_evolution.get("description", ""),
        "automated_tasks": role_evolution.get("automated_tasks", []),
        "augmented_tasks": role_evolution.get("augmented_tasks", []),
        "new_tasks": role_evolution.get("new_tasks", []),
        "new_skills_required": role_evolution.get("new_skills_required", []),
        "affected_employees": affected_employees,
        "readiness_score": readiness_score,
        "implementation_priority": implementation_priority,
        "recommendations": recommendations
    }

    return analysis


def analyze_all_role_evolutions(
    role_evolution_data: List[Dict[str, Any]],
    employees_by_role: Dict[str, List[Dict[str, Any]]],
    roles_lookup: Dict[str, Dict[str, Any]],
    automation_risk_analysis: List[Dict[str, Any]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> List[Dict[str, Any]]:
    """Analyze all role evolution opportunities"""
    analyses = []

    for evolution in role_evolution_data:
        analysis = analyze_role_evolution(
            evolution,
            employees_by_role,
            roles_lookup,
            automation_risk_analysis,
            config
        )
        analyses.append(analysis)

    return analyses



# Test role evolution analysis utilities

In [ ]:
"""Test role evolution analysis utilities

Testing Phase 6: Role Evolution Analysis Utilities
Following the pattern: Test utilities before building nodes
"""

from pathlib import Path
from agents.workforce_development_orchestrator.utilities.role_evolution import (
    calculate_role_readiness_score,
    analyze_role_evolution,
    analyze_all_role_evolutions
)
from agents.workforce_development_orchestrator.utilities.data_loading import (
    load_role_evolution,
    load_roles,
    load_employees,
    build_roles_lookup,
    build_employees_by_role
)
from agents.workforce_development_orchestrator.utilities.automation_risk import (
    analyze_all_roles_automation_risk
)
from agents.workforce_development_orchestrator.utilities.data_loading import (
    load_tasks,
    build_tasks_by_role
)
from config import WorkforceDevelopmentOrchestratorConfig


def test_calculate_role_readiness_score():
    """Test calculating role readiness score"""
    data_dir = Path("agents/data")

    # Load data
    role_evolution_data = load_role_evolution(data_dir)
    employees = load_employees(data_dir)
    employees_by_role = build_employees_by_role(employees)
    roles = load_roles(data_dir)
    tasks = load_tasks(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)

    config = WorkforceDevelopmentOrchestratorConfig()

    # Analyze automation risk
    automation_risk_analysis = analyze_all_roles_automation_risk(
        roles,
        tasks_by_role,
        employees_by_role,
        config
    )

    # Test with first evolution
    evolution = role_evolution_data[0]
    score = calculate_role_readiness_score(
        evolution,
        employees_by_role,
        automation_risk_analysis
    )

    assert 0.0 <= score <= 1.0

    print("✅ test_calculate_role_readiness_score: PASSED")


def test_analyze_role_evolution():
    """Test analyzing a single role evolution"""
    data_dir = Path("agents/data")

    # Load data
    role_evolution_data = load_role_evolution(data_dir)
    employees = load_employees(data_dir)
    roles = load_roles(data_dir)
    tasks = load_tasks(data_dir)
    employees_by_role = build_employees_by_role(employees)
    roles_lookup = build_roles_lookup(roles)
    tasks_by_role = build_tasks_by_role(tasks)

    config = WorkforceDevelopmentOrchestratorConfig()

    # Analyze automation risk
    automation_risk_analysis = analyze_all_roles_automation_risk(
        roles,
        tasks_by_role,
        employees_by_role,
        config
    )

    # Test with first evolution
    evolution = role_evolution_data[0]
    analysis = analyze_role_evolution(
        evolution,
        employees_by_role,
        roles_lookup,
        automation_risk_analysis,
        config
    )

    assert "role_id" in analysis
    assert "evolution_id" in analysis
    assert "readiness_score" in analysis
    assert "implementation_priority" in analysis
    assert "affected_employees" in analysis
    assert "recommendations" in analysis

    print("✅ test_analyze_role_evolution: PASSED")


def test_analyze_all_role_evolutions():
    """Test analyzing all role evolutions"""
    data_dir = Path("agents/data")

    # Load data
    role_evolution_data = load_role_evolution(data_dir)
    employees = load_employees(data_dir)
    roles = load_roles(data_dir)
    tasks = load_tasks(data_dir)
    employees_by_role = build_employees_by_role(employees)
    roles_lookup = build_roles_lookup(roles)
    tasks_by_role = build_tasks_by_role(tasks)

    config = WorkforceDevelopmentOrchestratorConfig()

    # Analyze automation risk
    automation_risk_analysis = analyze_all_roles_automation_risk(
        roles,
        tasks_by_role,
        employees_by_role,
        config
    )

    analyses = analyze_all_role_evolutions(
        role_evolution_data,
        employees_by_role,
        roles_lookup,
        automation_risk_analysis,
        config
    )

    assert len(analyses) == len(role_evolution_data)
    assert all("readiness_score" in a for a in analyses)
    assert all("implementation_priority" in a for a in analyses)

    print("✅ test_analyze_all_role_evolutions: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Role Evolution Analysis Utilities (Phase 6)")
    print("=" * 60)
    print()

    test_calculate_role_readiness_score()
    test_analyze_role_evolution()
    test_analyze_all_role_evolutions()

    print()
    print("=" * 60)
    print("✅ All role evolution utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_role_evolution_utilities.py
============================================================
Testing Role Evolution Analysis Utilities (Phase 6)
============================================================

✅ test_calculate_role_readiness_score: PASSED
✅ test_analyze_role_evolution: PASSED
✅ test_analyze_all_role_evolutions: PASSED

============================================================
✅ All role evolution utility tests passed!
============================================================
